In [1]:
import os
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import tensorboard

%matplotlib inline

In [1]:
from graph_bridges.models.backward_rates.backward_rate import GaussianTargetRateImageX0PredEMA
from graph_bridges.data.dataloaders import DoucetTargetData, GraphSpinsDataLoader
from graph_bridges.models.samplers.sampling import ReferenceProcess
from graph_bridges.models.reference_process.ctdd_reference import ReferenceProcess
from graph_bridges.models.losses.ctdd_losses import GenericAux
from graph_bridges.models.schedulers.scheduling_ctdd import CTDDScheduler
from graph_bridges.data.dataloaders_config import GraphSpinsDataLoaderConfig

    
from graph_bridges.models.pipelines.pipelines_utils import create_pipelines
from graph_bridges.models.schedulers.scheduling_utils import create_scheduler
from graph_bridges.models.backward_rates.backward_rate_utils import create_model
from graph_bridges.models.reference_process.reference_process_utils import create_reference
from graph_bridges.data.dataloaders_utils import create_dataloader
from graph_bridges.models.losses.loss_utils import create_loss

from graph_bridges.configs.graphs.lobster.config_mlp import BridgeMLPConfig
from graph_bridges.configs.graphs.lobster.config_base import BridgeConfig

In [2]:
mlp_config = BridgeMLPConfig()
config = BridgeConfig()
config.data = GraphSpinsDataLoaderConfig()
config.model = mlp_config.model
device = torch.device(config.device)

NameError: name 'torch' is not defined

In [ ]:
#=================================================================
# CREATE OBJECTS FROM CONFIGURATION

data_dataloader: GraphSpinsDataLoader
model : GaussianTargetRateImageX0PredEMA
reference_process: ReferenceProcess
loss : GenericAux
scheduler:CTDDScheduler

data_dataloader = create_dataloader(config,device)
model = create_model(config,device)
reference_process = create_reference(config,device)
loss = create_loss(config,device)
scheduler = create_scheduler(config,device)

In [ ]:
#=================================================================
minibatch = next(data_dataloader.train().__iter__())[0]
#minibatch = sample_.unsqueeze(1).unsqueeze(1)

In [6]:
# TIME ===========================================================
#ts = torch.rand((minibatch.shape[0],), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
B = minibatch.shape[0]
ts = torch.rand((B,), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
#==========

x_t, x_tilde, qt0, rate = scheduler.add_noise(minibatch,reference_process,ts,device,return_dict=False)
scheduler_noise_output = scheduler.add_noise(minibatch,reference_process,ts,device,return_dict=True)

x_logits,p0t_reg,p0t_sig,reg_x = model.forward(minibatch,ts,x_tilde)
model_forward_output = model.forward(minibatch,ts,x_tilde,return_dict=True)

loss_ = loss.calc_loss(minibatch,x_tilde,qt0,rate,x_logits,reg_x,p0t_sig,p0t_reg,device)

In [7]:
config.initialize()

In [8]:
config.tensorboard_path

'C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\Codes\\graph-bridges\\results\\graph\\lobster\\testing\\tensorboard'

In [9]:
#config.best_model_path
#config.sinkhorn_plot_path

In [10]:
config.optimizer.learning_rate = 0.01
config.optimizer.learning_rate

0.01

In [11]:
from diffusers.optimization import get_cosine_schedule_with_warmup
optimizer = torch.optim.AdamW(model.parameters(), lr=config.optimizer.learning_rate)
#lr_scheduler = get_cosine_schedule_with_warmup(
#    optimizer=optimizer,
#    num_warmup_steps=config.optimizer.lr_warmup_steps,
#    num_training_steps=(num_batches * config.optimizer.num_epochs),
#)

In [12]:
loss_ = loss.calc_loss(minibatch,x_tilde,qt0,rate,x_logits,reg_x,p0t_sig,p0t_reg,device)

In [13]:
#logs_dir = Path(save_dir).joinpath('tensorboard')
#logs_dir.mkdir(exist_ok=True)


#writer = SummaryWriter(logs_dir)


In [3]:
config.sampler.num_steps

1000

In [ ]:
        # Sample a random timestep for each image
        ts = torch.rand((B,), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
        

In [14]:
config.initialize()
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(config.tensorboard_path)
global_step = 0

# Now you train the model
for epoch in range(config.optimizer.num_epochs):
    progress_bar = tqdm(total=len(data_dataloader.train()))
    progress_bar.set_description(f"Epoch {epoch}")

    for step,minibatch in enumerate(data_dataloader.train()):
        minibatch = minibatch[0]
        B = minibatch.shape[0]
        
        # Sample a random timestep for each image
        ts = torch.rand((B,), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
        
        # Sample noise to add to the images
        # Add noise to the clean images according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        
        #noisy_images = scheduler.add_noise(clean_images, noise, timesteps)

        x_t, x_tilde, qt0, rate = scheduler.add_noise(minibatch,
                                                      reference_process,
                                                      ts,
                                                      device,
                                                      return_dict=False)
        
        #scheduler_noise_output = scheduler.add_noise(minibatch,
        #                                             reference_process,
        #                                             ts,
        #                                             device,
        #                                             return_dict=True)
        # Predict the noise residual
        #noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
        #loss = F.mse_loss(noise_pred, noise)
        #accelerator.backward(loss)
            
        x_logits,p0t_reg,p0t_sig,reg_x = model.forward(minibatch,
                                                       ts,
                                                       x_tilde)
            
        #model_forward_output = model.forward(minibatch,
        #                                     ts,
        #                                     x_tilde,
        #                                     return_dict=True)
            
        loss_ = loss.calc_loss(minibatch,
                               x_tilde,
                               qt0,
                               rate,
                               x_logits,
                               reg_x,
                               p0t_sig,
                               p0t_reg,
                               device)
        
        loss_.backward()
        optimizer.step()
        #lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        logs = {"loss": loss_.detach().item(), 
                #"lr": lr_scheduler.get_last_lr()[0], 
                "step": global_step}
        writer.add_scalar("loss",loss_.detach().item(),global_step)
        
        progress_bar.set_postfix(**logs)
        #accelerator.log(logs, step=global_step)
        global_step += 1
writer.close()

Epoch 0: 100%|██████████████████████████████████████████████████████| 25/25 [00:00<00:00, 105.03it/s, loss=86.1, step=24]

Epoch 2: 100%|███████████████████████████████████████████████████████| 25/25 [00:00<00:00, 98.01it/s, loss=84.7, step=74]

Epoch 4: 100%|██████████████████████████████████████████████████████| 25/25 [00:00<00:00, 98.97it/s, loss=78.5, step=124]

Epoch 6: 100%|██████████████████████████████████████████████████████| 25/25 [00:00<00:00, 86.55it/s, loss=80.7, step=174]

Epoch 8: 100%|██████████████████████████████████████████████████████| 25/25 [00:00<00:00, 79.19it/s, loss=89.9, step=224]

Epoch 9:  16%|████████▊                                              | 4/25 [00:00<00:00, 62.28it/s, loss=88.6, step=228]


Epoch 10: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 86.97it/s, loss=96.9, step=274]

Epoch 12: 100%|██████████████████████████████████████████████████████| 25/25 [00:00<00:00, 90.69it/s, loss=109, step=324]

Epoch 14: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 94.00it/s, loss=88.3, step=374]

Epoch 16: 100%|███████████████████████████████████████████████████████| 25/25 [00:00<00:00, 93.47it/s, loss=91, step=424]

Epoch 17:  44%|███████████████████████▎                             | 11/25 [00:00<00:00, 73.07it/s, loss=89.8, step=435]


Epoch 18: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 84.93it/s, loss=84.2, step=474]

Epoch 20: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 84.85it/s, loss=79.9, step=524]

Epoch 22: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 99.78it/s, loss=99.9, step=574]

Epoch 24: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 100.08it/s, loss=103, step=624]

Epoch 25:  72%|██████████████████████████████████████▏              | 18/25 [00:00<00:00, 82.39it/s, loss=91.1, step=642]


Epoch 26: 100%|████████████████████████████████████████████████████| 25/25 [00:00<00:00, 101.01it/s, loss=89.5, step=674]

Epoch 28: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 89.73it/s, loss=94.4, step=724]

Epoch 30: 100%|███████████████████████████████████████████████████████| 25/25 [00:00<00:00, 91.96it/s, loss=86, step=774]

Epoch 32: 100%|███████████████████████████████████████████████████████| 25/25 [00:00<00:00, 81.74it/s, loss=94, step=824]

Epoch 33: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 53.52it/s, loss=99.5, step=849]


Epoch 34: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 85.11it/s, loss=93.4, step=874]

Epoch 36: 100%|███████████████████████████████████████████████████████| 25/25 [00:00<00:00, 99.13it/s, loss=89, step=924]

Epoch 38: 100%|██████████████████████████████████████████████████████| 25/25 [00:00<00:00, 94.35it/s, loss=104, step=974]

Epoch 40: 100%|███████████████████████████████████████████████████| 25/25 [00:00<00:00, 100.32it/s, loss=90.7, step=1024]

Epoch 42: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 104.36it/s, loss=84, step=1074]

Epoch 43:  24%|████████████▋                                        | 6/25 [00:00<00:00, 81.45it/s, loss=83.8, step=1080]


Epoch 44: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 94.16it/s, loss=113, step=1124]

Epoch 46: 100%|█████████████████████████████████████████████████████| 25/25 [00:00<00:00, 95.16it/s, loss=101, step=1174]

Epoch 48: 100%|███████████████████████████████████████████████████| 25/25 [00:00<00:00, 100.57it/s, loss=86.8, step=1224]

Epoch 49: 100%|█████████████████████████████████████████████████████| 25/25 [00:14<00:00, 84.17it/s, loss=101, step=1249]